# 数据预处理

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
df = pd.read_csv('data.csv')
df = df.dropna(axis=0)
df

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,dt,kdcode,volume,turnover,turn,pct_chg,vwap,trade_status,adj_close,adj_open,adj_high,adj_low,label
0,2016-01-04,000001.SZ,56349787.0,6.603762e+08,0.4774,-5.51,11.719231,0,935.197763,990.500720,992.976972,926.943591,0.0
1,2016-01-05,000001.SZ,66326995.0,7.555314e+08,0.5619,0.61,11.391008,0,940.975684,930.245260,955.007778,920.340253,1.0
2,2016-01-06,000001.SZ,51570644.0,5.916985e+08,0.4369,1.14,11.473553,0,951.706109,942.626519,954.182361,940.150267,0.0
3,2016-01-07,000001.SZ,17476110.0,1.948695e+08,0.1481,-5.12,11.150622,0,903.006490,941.801102,941.801102,900.530238,0.0
4,2016-01-08,000001.SZ,74752758.0,8.313345e+08,0.6333,1.64,11.121122,0,917.864001,925.292756,931.896094,899.704821,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680759,2021-07-26,688981.SH,97665400.0,5.346218e+09,5.5900,10.25,54.740148,0,57.200000,51.850000,57.320000,51.120000,1.0
4680760,2021-07-27,688981.SH,308012900.0,1.993186e+10,16.4600,12.59,64.711114,0,64.400000,57.400000,68.640000,57.400000,1.0
4680761,2021-07-28,688981.SH,172850500.0,1.044512e+10,9.2400,-7.98,60.428625,0,59.260000,62.950000,63.490000,58.280000,0.0
4680762,2021-07-29,688981.SH,144274400.0,8.933641e+09,7.7100,5.08,61.921177,0,62.270000,60.400000,64.580000,59.370000,1.0


## 1. 根据股票代码划分数据

In [2]:
stocks_code = df['kdcode'].unique()
stocks_num = len(stocks_code)
"股票数量",stocks_num

('股票数量', 4022)

In [3]:
# 根据股票代码划分数据
for i, stock_i in enumerate(tqdm(stocks_code)):
    stock_i_data = df[df['kdcode'].isin([stock_i])]
    exec("df%s = stock_i_data" % i)

100%|██████████| 4022/4022 [04:04<00:00, 16.47it/s]


In [4]:
res_df = pd.DataFrame()

## 2. 按照股票添加特征

### 2.1 添加1日、5日、10日、20日`vwap`均价特征

In [5]:
sizes = [5, 10, 20]
res_df['vwap'] = df['vwap']

In [6]:
for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        vwap = stock['vwap'].values # 成交价
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = vwap[:index + 1].mean()
            else:
                mean_res = vwap[index - size + 1:index + 1].mean()
            exec("meanList%s.append(%s)" % (t, mean_res))

res_df['vwap_mean5'] = meanList0
res_df['vwap_mean10'] = meanList1
res_df['vwap_mean20'] = meanList2

遍历所有股票: 100%|██████████| 4022/4022 [01:39<00:00, 40.40it/s] 


### 2.2 添加3日、5日、10日平均涨跌幅`pct_chg`

In [7]:
sizes = [3, 5, 10]
for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        pct_chg = stock['pct_chg'].values # 涨跌幅
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = pct_chg[:index + 1].mean()
            else:
                mean_res = pct_chg[index - size + 1:index + 1].mean()
            exec("meanList%s.append(%s)" % (t, mean_res))

res_df['chg_mean3'] = meanList0
res_df['chg_mean5'] = meanList1
res_df['chg_mean10'] = meanList2

遍历所有股票: 100%|██████████| 4022/4022 [01:26<00:00, 46.72it/s] 


In [8]:
res_df

,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10
0,11.719231,11.719231,11.719231,11.719231,-5.510000,-5.510000,-5.510000
1,11.391008,11.555119,11.555119,11.555119,-2.450000,-2.450000,-2.450000
2,11.473553,11.527931,11.527931,11.527931,-1.253333,-1.253333,-1.253333
3,11.150622,11.433604,11.433604,11.433604,-1.123333,-2.220000,-2.220000
4,11.121122,11.371107,11.371107,11.371107,-0.780000,-1.448000,-1.448000
...,...,...,...,...,...,...,...
4680759,54.740148,52.787750,53.961543,56.229254,2.966667,2.088000,0.170000
4680760,64.711114,55.416810,54.720099,56.497093,6.910000,4.576000,1.353000
4680761,60.428625,57.019739,55.101627,56.441425,4.953333,2.702000,0.737000
4680762,61.921177,58.804745,55.757486,56.499986,3.230000,3.566000,1.393000


### 2.3 添加3日、5日、10日平均开盘价

In [9]:
sizes = [3, 5, 10]
for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        adj_open = stock['adj_open'].values # 开盘价
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = adj_open[:index + 1].mean()
            else:
                mean_res = adj_open[index - size + 1:index + 1].mean()
            exec("meanList%s.append(%s)" % (t, mean_res))

res_df['open_mean3'] = meanList0
res_df['open_mean5'] = meanList1
res_df['open_mean10'] = meanList2

遍历所有股票: 100%|██████████| 4022/4022 [01:29<00:00, 45.01it/s] 


In [10]:
res_df

,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10
0,11.719231,11.719231,11.719231,11.719231,-5.510000,-5.510000,-5.510000,990.500720,990.500720,990.500720
1,11.391008,11.555119,11.555119,11.555119,-2.450000,-2.450000,-2.450000,960.372990,960.372990,960.372990
2,11.473553,11.527931,11.527931,11.527931,-1.253333,-1.253333,-1.253333,954.457500,954.457500,954.457500
3,11.150622,11.433604,11.433604,11.433604,-1.123333,-2.220000,-2.220000,938.224293,951.293400,951.293400
4,11.121122,11.371107,11.371107,11.371107,-0.780000,-1.448000,-1.448000,936.573459,946.093271,946.093271
...,...,...,...,...,...,...,...,...,...,...
4680759,54.740148,52.787750,53.961543,56.229254,2.966667,2.088000,0.170000,52.496667,52.078000,53.945000
4680760,64.711114,55.416810,54.720099,56.497093,6.910000,4.576000,1.353000,54.083333,53.358000,54.024000
4680761,60.428625,57.019739,55.101627,56.441425,4.953333,2.702000,0.737000,57.400000,55.568000,54.614000
4680762,61.921177,58.804745,55.757486,56.499986,3.230000,3.566000,1.393000,60.250000,57.120000,55.052000


### 2.4 添加1、3、5、10日的顺势指标CCI

In [11]:
sizes = [1, 3, 5, 10]

def getCCI(data, ndays):
    TP = (data['adj_high'] + data['adj_low'] + data['adj_close']) / 3 
    CCI = pd.Series((TP - TP.rolling(ndays).mean()) / (0.015 * TP.rolling(ndays).mean()),name = 'CCI')
    return CCI

for t, size in enumerate(sizes):
    CCISeries = pd.Series()
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        CCI = getCCI(stock, size)
        for index in range(size - 1):
            cci = getCCI(stock.iloc[:index + 1], index + 1).iloc[index]
            CCI.iloc[index] = cci
        CCISeries = pd.concat([CCISeries, CCI])
    print(CCISeries)
    print("max: ", CCISeries.max(), "min: ", CCISeries.min())
    res_df['CCI_' + str(size)] = CCISeries
    
res_df

遍历所有股票: 100%|██████████| 4022/4022 [02:23<00:00, 28.03it/s]


0          0.000000e+00
1          0.000000e+00
2          0.000000e+00
3          0.000000e+00
4          0.000000e+00
               ...     
4680759   -3.431745e-14
4680760   -2.984847e-14
4680761   -2.355000e-14
4680762   -2.289365e-14
4680763   -1.478372e-14
Length: 4680764, dtype: float64
max:  5.741930085494412e-12 min:  -9.616325693090244e-12


遍历所有股票: 100%|██████████| 4022/4022 [01:34<00:00, 42.39it/s] 


0          0.000000
1         -0.456023
2          0.161514
3         -1.361346
4         -0.738908
             ...   
4680759    2.173597
4680760    7.612250
4680761    0.742244
4680762    0.115955
4680763    2.055407
Length: 4680764, dtype: float64
max:  110.5888782040493 min:  -60.7621804148577


遍历所有股票: 100%|██████████| 4022/4022 [01:40<00:00, 39.97it/s]


0          0.000000
1         -0.456023
2          0.161514
3         -1.666056
4         -1.260603
             ...   
4680759    2.968886
4680760    9.930618
4680761    4.100700
4680762    3.869609
4680763    3.325397
Length: 4680764, dtype: float64
max:  198.59327701757957 min:  -61.67254210732472


遍历所有股票: 100%|██████████| 4022/4022 [02:16<00:00, 29.42it/s]


0           0.000000
1          -0.456023
2           0.161514
3          -1.666056
4          -1.260603
             ...    
4680759     1.477751
4680760    10.760715
4680761     6.407679
4680762     7.609942
4680763     8.750333
Length: 4680764, dtype: float64
max:  355.965663392702 min:  -61.95883187289753


,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10,CCI_1,CCI_3,CCI_5,CCI_10
0,11.719231,11.719231,11.719231,11.719231,-5.510000,-5.510000,-5.510000,990.500720,990.500720,990.500720,0.000000e+00,0.000000,0.000000,0.000000
1,11.391008,11.555119,11.555119,11.555119,-2.450000,-2.450000,-2.450000,960.372990,960.372990,960.372990,0.000000e+00,-0.456023,-0.456023,-0.456023
2,11.473553,11.527931,11.527931,11.527931,-1.253333,-1.253333,-1.253333,954.457500,954.457500,954.457500,0.000000e+00,0.161514,0.161514,0.161514
3,11.150622,11.433604,11.433604,11.433604,-1.123333,-2.220000,-2.220000,938.224293,951.293400,951.293400,0.000000e+00,-1.361346,-1.666056,-1.666056
4,11.121122,11.371107,11.371107,11.371107,-0.780000,-1.448000,-1.448000,936.573459,946.093271,946.093271,0.000000e+00,-0.738908,-1.260603,-1.260603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680759,54.740148,52.787750,53.961543,56.229254,2.966667,2.088000,0.170000,52.496667,52.078000,53.945000,-3.431745e-14,2.173597,2.968886,1.477751
4680760,64.711114,55.416810,54.720099,56.497093,6.910000,4.576000,1.353000,54.083333,53.358000,54.024000,-2.984847e-14,7.612250,9.930618,10.760715
4680761,60.428625,57.019739,55.101627,56.441425,4.953333,2.702000,0.737000,57.400000,55.568000,54.614000,-2.355000e-14,0.742244,4.100700,6.407679
4680762,61.921177,58.804745,55.757486,56.499986,3.230000,3.566000,1.393000,60.250000,57.120000,55.052000,-2.289365e-14,0.115955,3.869609,7.609942


### 2.5 添加1、3、5、10日的简易波动指标EVM

In [12]:
sizes = [1, 3, 5, 10]

def getEVM(data, ndays):
    dm = ((data['adj_high'] + data['adj_low'])/2) - ((data['adj_high'].shift(1) + data['adj_low'].shift(1))/2)
    br = (data['volume']) / ((data['adj_high'] - data['adj_low']))
    EVM = dm / br
    EVM_MA = pd.Series(EVM.rolling(ndays).mean(), name = 'EVM')
    return EVM_MA

for t, size in enumerate(sizes):
    EVMSeries = pd.Series()
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        EVM = getEVM(stock, size)
        for index in range(size - 1):
            evm = getEVM(stock.iloc[:index + 1], index + 1).iloc[index]
            EVM.iloc[index] = evm
        EVMSeries = pd.concat([EVMSeries, EVM])
    print(EVMSeries)
    print("max: ", EVMSeries.max(), "min: ", EVMSeries.min())
    res_df['EVM_' + str(size)] = EVMSeries
        
res_df

遍历所有股票: 100%|██████████| 4022/4022 [01:18<00:00, 51.39it/s] 


0                   NaN
1         -1.164850e-05
2          2.582803e-06
3         -6.140205e-05
4         -2.310457e-06
               ...     
4680759    1.152199e-07
4680760    3.211294e-07
4680761   -6.435243e-08
4680762    3.936180e-08
4680763    8.180441e-08
Length: 4680764, dtype: float64
max:  4.871077404044781 min:  -15.35398166752208


遍历所有股票: 100%|██████████| 4022/4022 [01:25<00:00, 47.28it/s]


0                   NaN
1                   NaN
2                   NaN
3         -2.348925e-05
4         -2.037657e-05
               ...     
4680759    3.696239e-08
4680760    1.373619e-07
4680761    1.239990e-07
4680762    9.871292e-08
4680763    1.893792e-08
Length: 4680764, dtype: float64
max:  4.218348732717664 min:  -9.233552228325939


遍历所有股票: 100%|██████████| 4022/4022 [01:48<00:00, 37.22it/s]


0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
               ...     
4680759    2.109421e-08
4680760    9.408162e-08
4680761    7.353283e-08
4680762    7.741899e-08
4680763    9.863262e-08
Length: 4680764, dtype: float64
max:  2.3358424383600473 min:  -7.8872571098102995


遍历所有股票: 100%|██████████| 4022/4022 [02:16<00:00, 29.57it/s]


0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
               ...     
4680759   -5.093167e-09
4680760    2.407406e-08
4680761    2.019063e-08
4680762    2.833900e-08
4680763    3.839392e-08
Length: 4680764, dtype: float64
max:  0.8857420698086695 min:  -3.4456220720703783


,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10,CCI_1,CCI_3,CCI_5,CCI_10,EVM_1,EVM_3,EVM_5,EVM_10
0,11.719231,11.719231,11.719231,11.719231,-5.510000,-5.510000,-5.510000,990.500720,990.500720,990.500720,0.000000e+00,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
1,11.391008,11.555119,11.555119,11.555119,-2.450000,-2.450000,-2.450000,960.372990,960.372990,960.372990,0.000000e+00,-0.456023,-0.456023,-0.456023,-1.164850e-05,NaN,NaN,NaN
2,11.473553,11.527931,11.527931,11.527931,-1.253333,-1.253333,-1.253333,954.457500,954.457500,954.457500,0.000000e+00,0.161514,0.161514,0.161514,2.582803e-06,NaN,NaN,NaN
3,11.150622,11.433604,11.433604,11.433604,-1.123333,-2.220000,-2.220000,938.224293,951.293400,951.293400,0.000000e+00,-1.361346,-1.666056,-1.666056,-6.140205e-05,-2.348925e-05,NaN,NaN
4,11.121122,11.371107,11.371107,11.371107,-0.780000,-1.448000,-1.448000,936.573459,946.093271,946.093271,0.000000e+00,-0.738908,-1.260603,-1.260603,-2.310457e-06,-2.037657e-05,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680759,54.740148,52.787750,53.961543,56.229254,2.966667,2.088000,0.170000,52.496667,52.078000,53.945000,-3.431745e-14,2.173597,2.968886,1.477751,1.152199e-07,3.696239e-08,2.109421e-08,-5.093167e-09
4680760,64.711114,55.416810,54.720099,56.497093,6.910000,4.576000,1.353000,54.083333,53.358000,54.024000,-2.984847e-14,7.612250,9.930618,10.760715,3.211294e-07,1.373619e-07,9.408162e-08,2.407406e-08
4680761,60.428625,57.019739,55.101627,56.441425,4.953333,2.702000,0.737000,57.400000,55.568000,54.614000,-2.355000e-14,0.742244,4.100700,6.407679,-6.435243e-08,1.239990e-07,7.353283e-08,2.019063e-08
4680762,61.921177,58.804745,55.757486,56.499986,3.230000,3.566000,1.393000,60.250000,57.120000,55.052000,-2.289365e-14,0.115955,3.869609,7.609942,3.936180e-08,9.871292e-08,7.741899e-08,2.833900e-08


### 2.6 添加1、3、5、10日的强力指数FI

In [ ]:
# sizes = [1, 3, 5, 10]
# def getFI(data, ndays):
#     FI = pd.Series(data['adj_close'].diff(ndays) * data['volume'], name = 'ForceIndex') 
#     return FI

# for t, size in enumerate(sizes):
#     FISeries = pd.Series()
#     for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
#         exec("stock = df%s" % i)
#         FI = getFI(stock, size)
#         for index in range(size - 1):
#             fi = getFI(stock.iloc[:index + 1], index + 1).iloc[index]
#             FI.iloc[index] = fi
#         FISeries = pd.concat([FISeries, FI])
#     print(FISeries)
# #     FISeries = FISeries / 100000000
#     print("max: ", FISeries.max(), "min: ", FISeries.min())
#     res_df['FI_' + str(size)] = FISeries
# res_df

## 3. 特征处理

* **通过取对数log缩小数据差距**

open_mean的最小值与最大值差距过大

In [27]:
# res_df['open_mean3'].min()

1.0868

In [28]:
# res_df['open_mean3'].max()

237952.08

In [29]:
# res_df['open_mean3'] = np.log(res_df['open_mean3'])
# res_df['open_mean5'] = np.log(res_df['open_mean5'])
# res_df['open_mean10'] = np.log(res_df['open_mean10'])
# res_df

,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10
0,11.719231,11.719231,11.719231,11.719231,-5.510000,-5.510000,-5.510000,6.898211,6.898211,6.898211
1,11.391008,11.555119,11.555119,11.555119,-2.450000,-2.450000,-2.450000,6.867322,6.867322,6.867322
2,11.473553,11.527931,11.527931,11.527931,-1.253333,-1.253333,-1.253333,6.861143,6.861143,6.861143
3,11.150622,11.433604,11.433604,11.433604,-1.123333,-2.220000,-2.220000,6.843989,6.857823,6.857823
4,11.121122,11.371107,11.371107,11.371107,-0.780000,-1.448000,-1.448000,6.842228,6.852341,6.852341
...,...,...,...,...,...,...,...,...,...,...
4680759,54.740148,52.787750,53.961543,56.229254,2.966667,2.088000,0.170000,3.960750,3.952743,3.987965
4680760,64.711114,55.416810,54.720099,56.497093,6.910000,4.576000,1.353000,3.990526,3.977024,3.989428
4680761,60.428625,57.019739,55.101627,56.441425,4.953333,2.702000,0.737000,4.050044,4.017607,4.000290
4680762,61.921177,58.804745,55.757486,56.499986,3.230000,3.566000,1.393000,4.098503,4.045154,4.008278


In [32]:
# res_df['vwap'].max()

2554.307336

* **通过删除$3\sigma$外的数据**

## 4. 归一化

In [16]:
tmp_df = res_df.copy()

In [18]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
_df = min_max_scaler.fit_transform(res_df)
_df = pd.DataFrame(_df, columns=res_df.columns)
_df

,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10,...,CCI_3,CCI_5,CCI_10,EVM_1,EVM_3,EVM_5,EVM_10,FI_1,FI_3,FI_5
0,0.004588,0.004771,0.004929,0.005196,0.016312,0.016312,0.016312,0.004158,0.004158,0.004158,...,0.354606,0.236960,0.148254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.004460,0.004704,0.004860,0.005123,0.019133,0.019133,0.019133,0.004031,0.004031,0.004031,...,0.351945,0.235208,0.147162,0.759156,NaN,NaN,NaN,0.359981,NaN,NaN
2,0.004492,0.004693,0.004849,0.005111,0.020236,0.020236,0.020236,0.004007,0.004007,0.004006,...,0.355549,0.237580,0.148640,0.759156,NaN,NaN,NaN,0.360038,NaN,NaN
3,0.004365,0.004655,0.004809,0.005069,0.020356,0.019345,0.019345,0.003938,0.003993,0.003993,...,0.346662,0.230558,0.144267,0.759153,0.686411,NaN,NaN,0.359565,0.320444,NaN
4,0.004354,0.004629,0.004783,0.005042,0.020673,0.020057,0.020057,0.003931,0.003971,0.003971,...,0.350294,0.232116,0.145237,0.759156,0.686411,NaN,NaN,0.360226,0.320212,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680759,0.021431,0.021490,0.022697,0.024931,0.024127,0.023317,0.021548,0.000216,0.000214,0.000222,...,0.367291,0.248367,0.151790,0.759156,0.686412,0.771513,0.795505,0.360027,0.320647,0.318438
4680760,0.025334,0.022560,0.023016,0.025050,0.027763,0.025611,0.022639,0.000223,0.000220,0.000222,...,0.399031,0.275115,0.174002,0.759156,0.686412,0.771513,0.795505,0.360599,0.321258,0.318985
4680761,0.023658,0.023212,0.023177,0.025025,0.025959,0.023883,0.022071,0.000237,0.000229,0.000225,...,0.358938,0.252716,0.163586,0.759156,0.686412,0.771513,0.795505,0.359553,0.320812,0.318540
4680762,0.024242,0.023939,0.023453,0.025051,0.024370,0.024680,0.022676,0.000249,0.000235,0.000227,...,0.355283,0.251828,0.166463,0.759156,0.686412,0.771513,0.795505,0.359998,0.320703,0.318571


In [20]:
# 添加不需归一化的列
_df['label'] = df['label']
_df['kdcode'] = df['kdcode']
_df['dt'] = df['dt']
order = ['dt','kdcode','vwap','vwap_mean5','vwap_mean10','vwap_mean20','chg_mean3','chg_mean5','chg_mean10','open_mean3','open_mean5','open_mean10','CCI_1','CCI_3','CCI_5','CCI_10','EVM_1','EVM_3','EVM_5','EVM_10','label']
df_ = _df[order]
df_

,dt,kdcode,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,...,open_mean10,CCI_1,CCI_3,CCI_5,CCI_10,EVM_1,EVM_3,EVM_5,EVM_10,label
0,2016-01-04,000001.SZ,0.004588,0.004771,0.004929,0.005196,0.016312,0.016312,0.016312,0.004158,...,0.004158,0.626134,0.354606,0.236960,0.148254,NaN,NaN,NaN,NaN,0.0
1,2016-01-05,000001.SZ,0.004460,0.004704,0.004860,0.005123,0.019133,0.019133,0.019133,0.004031,...,0.004031,0.626134,0.351945,0.235208,0.147162,0.759156,NaN,NaN,NaN,1.0
2,2016-01-06,000001.SZ,0.004492,0.004693,0.004849,0.005111,0.020236,0.020236,0.020236,0.004007,...,0.004006,0.626134,0.355549,0.237580,0.148640,0.759156,NaN,NaN,NaN,0.0
3,2016-01-07,000001.SZ,0.004365,0.004655,0.004809,0.005069,0.020356,0.019345,0.019345,0.003938,...,0.003993,0.626134,0.346662,0.230558,0.144267,0.759153,0.686411,NaN,NaN,0.0
4,2016-01-08,000001.SZ,0.004354,0.004629,0.004783,0.005042,0.020673,0.020057,0.020057,0.003931,...,0.003971,0.626134,0.350294,0.232116,0.145237,0.759156,0.686411,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680759,2021-07-26,688981.SH,0.021431,0.021490,0.022697,0.024931,0.024127,0.023317,0.021548,0.000216,...,0.000222,0.623900,0.367291,0.248367,0.151790,0.759156,0.686412,0.771513,0.795505,1.0
4680760,2021-07-27,688981.SH,0.025334,0.022560,0.023016,0.025050,0.027763,0.025611,0.022639,0.000223,...,0.000222,0.624190,0.399031,0.275115,0.174002,0.759156,0.686412,0.771513,0.795505,1.0
4680761,2021-07-28,688981.SH,0.023658,0.023212,0.023177,0.025025,0.025959,0.023883,0.022071,0.000237,...,0.000225,0.624601,0.358938,0.252716,0.163586,0.759156,0.686412,0.771513,0.795505,0.0
4680762,2021-07-29,688981.SH,0.024242,0.023939,0.023453,0.025051,0.024370,0.024680,0.022676,0.000249,...,0.000227,0.624643,0.355283,0.251828,0.166463,0.759156,0.686412,0.771513,0.795505,1.0


In [21]:
# df_ = df_.dropna(axis=0)
df_ = df_.fillna(0)
df_

,dt,kdcode,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,...,open_mean10,CCI_1,CCI_3,CCI_5,CCI_10,EVM_1,EVM_3,EVM_5,EVM_10,label
0,2016-01-04,000001.SZ,0.004588,0.004771,0.004929,0.005196,0.016312,0.016312,0.016312,0.004158,...,0.004158,0.626134,0.354606,0.236960,0.148254,0.000000,0.000000,0.000000,0.000000,0.0
1,2016-01-05,000001.SZ,0.004460,0.004704,0.004860,0.005123,0.019133,0.019133,0.019133,0.004031,...,0.004031,0.626134,0.351945,0.235208,0.147162,0.759156,0.000000,0.000000,0.000000,1.0
2,2016-01-06,000001.SZ,0.004492,0.004693,0.004849,0.005111,0.020236,0.020236,0.020236,0.004007,...,0.004006,0.626134,0.355549,0.237580,0.148640,0.759156,0.000000,0.000000,0.000000,0.0
3,2016-01-07,000001.SZ,0.004365,0.004655,0.004809,0.005069,0.020356,0.019345,0.019345,0.003938,...,0.003993,0.626134,0.346662,0.230558,0.144267,0.759153,0.686411,0.000000,0.000000,0.0
4,2016-01-08,000001.SZ,0.004354,0.004629,0.004783,0.005042,0.020673,0.020057,0.020057,0.003931,...,0.003971,0.626134,0.350294,0.232116,0.145237,0.759156,0.686411,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680759,2021-07-26,688981.SH,0.021431,0.021490,0.022697,0.024931,0.024127,0.023317,0.021548,0.000216,...,0.000222,0.623900,0.367291,0.248367,0.151790,0.759156,0.686412,0.771513,0.795505,1.0
4680760,2021-07-27,688981.SH,0.025334,0.022560,0.023016,0.025050,0.027763,0.025611,0.022639,0.000223,...,0.000222,0.624190,0.399031,0.275115,0.174002,0.759156,0.686412,0.771513,0.795505,1.0
4680761,2021-07-28,688981.SH,0.023658,0.023212,0.023177,0.025025,0.025959,0.023883,0.022071,0.000237,...,0.000225,0.624601,0.358938,0.252716,0.163586,0.759156,0.686412,0.771513,0.795505,0.0
4680762,2021-07-29,688981.SH,0.024242,0.023939,0.023453,0.025051,0.024370,0.024680,0.022676,0.000249,...,0.000227,0.624643,0.355283,0.251828,0.166463,0.759156,0.686412,0.771513,0.795505,1.0


## 5. 分开训练集和测试集

In [22]:
df_.set_index(['dt'], inplace = True)
df_.index = pd.to_datetime(df_.index)
df_

,kdcode,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10,CCI_1,CCI_3,CCI_5,CCI_10,EVM_1,EVM_3,EVM_5,EVM_10,label
dt,,,,,,,,,,,,,,,,,,,,
2016-01-04,000001.SZ,0.004588,0.004771,0.004929,0.005196,0.016312,0.016312,0.016312,0.004158,0.004158,0.004158,0.626134,0.354606,0.236960,0.148254,0.000000,0.000000,0.000000,0.000000,0.0
2016-01-05,000001.SZ,0.004460,0.004704,0.004860,0.005123,0.019133,0.019133,0.019133,0.004031,0.004031,0.004031,0.626134,0.351945,0.235208,0.147162,0.759156,0.000000,0.000000,0.000000,1.0
2016-01-06,000001.SZ,0.004492,0.004693,0.004849,0.005111,0.020236,0.020236,0.020236,0.004007,0.004007,0.004006,0.626134,0.355549,0.237580,0.148640,0.759156,0.000000,0.000000,0.000000,0.0
2016-01-07,000001.SZ,0.004365,0.004655,0.004809,0.005069,0.020356,0.019345,0.019345,0.003938,0.003993,0.003993,0.626134,0.346662,0.230558,0.144267,0.759153,0.686411,0.000000,0.000000,0.0
2016-01-08,000001.SZ,0.004354,0.004629,0.004783,0.005042,0.020673,0.020057,0.020057,0.003931,0.003971,0.003971,0.626134,0.350294,0.232116,0.145237,0.759156,0.686411,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,688981.SH,0.021431,0.021490,0.022697,0.024931,0.024127,0.023317,0.021548,0.000216,0.000214,0.000222,0.623900,0.367291,0.248367,0.151790,0.759156,0.686412,0.771513,0.795505,1.0
2021-07-27,688981.SH,0.025334,0.022560,0.023016,0.025050,0.027763,0.025611,0.022639,0.000223,0.000220,0.000222,0.624190,0.399031,0.275115,0.174002,0.759156,0.686412,0.771513,0.795505,1.0
2021-07-28,688981.SH,0.023658,0.023212,0.023177,0.025025,0.025959,0.023883,0.022071,0.000237,0.000229,0.000225,0.624601,0.358938,0.252716,0.163586,0.759156,0.686412,0.771513,0.795505,0.0


In [23]:
dateSepVal = pd.to_datetime("2020-01-01")
train_df = df_[df_.index < dateSepVal]
test_df = df_[df_.index > dateSepVal]

## 6. 保存到文件

In [24]:
train_df.to_csv('trainData.csv')

In [25]:
test_df.to_csv('testData.csv')